In [2]:
import findspark
findspark.init('/home/ronit/spark-3.0.0-preview2-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('lrex').getOrCreate()

In [10]:
from pyspark.ml.regression import LinearRegression

In [6]:
training = spark.read.format("libsvm").load('sample_linear_regression_data.txt')

In [8]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [12]:
lr = LinearRegression(featuresCol = 'features', labelCol='label', predictionCol='prediction')

In [13]:
lrModel = lr.fit(training)

In [14]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [15]:
lrModel.intercept

0.14228558260358093

In [16]:
training_summery = lrModel.summary

In [18]:
training_summery.rootMeanSquaredError

10.16309157133015

In [19]:
all_data = spark.read.format('libsvm').load("sample_linear_regression_data.txt")

In [20]:
split_object = all_data.randomSplit([0.7,0.3])
split_object

[DataFrame[label: double, features: vector],
 DataFrame[label: double, features: vector]]

In [21]:
train_data, test_data = all_data.randomSplit([0.7,0.3])

In [22]:
correct_model = lr.fit(train_data)

In [28]:
test_results = correct_model.evaluate(test_data)

In [29]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-26.828703180361465|
| -20.20187029616396|
|-21.326623783477903|
|-22.154805520041766|
|-14.428816428990622|
|-18.878776869866027|
|-19.050223396366462|
| -15.79781741568549|
|-14.537482306394121|
|-18.291360011740355|
| -16.43344507423695|
|-17.216991351212812|
|-14.919883318018268|
| -12.89030171772518|
|-12.256693192221233|
|-11.807738847672153|
|-14.783195023440147|
|-17.710291513891296|
|  -8.68109903935406|
|-13.103973893038448|
+-------------------+
only showing top 20 rows



In [30]:
test_redults.rootMeanSquaredError

9.812544613224

In [31]:
unlabeled_data = test_data.select('features')

In [32]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [33]:
prediction = correct_model.transform(unlabeled_data)

In [34]:
prediction.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...|   3.878877244165393|
|(10,[0,1,2,3,4,5,...|   0.317309521890537|
|(10,[0,1,2,3,4,5,...|  1.5438609938633643|
|(10,[0,1,2,3,4,5,...|   3.308883047143182|
|(10,[0,1,2,3,4,5,...|  -3.065383927892722|
|(10,[0,1,2,3,4,5,...|  1.4501022989265206|
|(10,[0,1,2,3,4,5,...|  2.7887931209137324|
|(10,[0,1,2,3,4,5,...|-0.35353193559162155|
|(10,[0,1,2,3,4,5,...| -1.4140302594004526|
|(10,[0,1,2,3,4,5,...|    2.55927173950111|
|(10,[0,1,2,3,4,5,...|  1.0575873509246518|
|(10,[0,1,2,3,4,5,...|  1.8574464713801357|
|(10,[0,1,2,3,4,5,...| 0.09773040826707832|
|(10,[0,1,2,3,4,5,...| -1.8724565352059461|
|(10,[0,1,2,3,4,5,...| -1.6103947029375352|
|(10,[0,1,2,3,4,5,...| -1.6128559282186044|
|(10,[0,1,2,3,4,5,...|  1.8053462980480441|
|(10,[0,1,2,3,4,5,...|   4.788068410520876|
|(10,[0,1,2,3,4,5,...|  -4.092127959897137|
|(10,[0,1,2,3,4,5,...|  0.603200

# realistic example

In [35]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [36]:
data = spark.read.csv("Ecommerce_Customers.csv", header = True, inferSchema=True)

In [37]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [42]:
for item in data.head(1)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [43]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [44]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [45]:
assembler = VectorAssembler(inputCols=['Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership'],outputCol="features")

In [46]:
output = assembler.transform(data)

In [47]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [50]:
type(output)

pyspark.sql.dataframe.DataFrame

In [54]:
output.select("features").head(1)

[Row(features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [56]:
final_data = output.select('features','Yearly Amount Spent')

In [57]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [58]:
training_data, test_data = final_data.randomSplit([0.7,0.3])

In [60]:
train_data.describe().show()


+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                335|
|   mean|0.22656261047461654|
| stddev| 10.699351174952586|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [61]:
test_data.describe().show()


+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                158|
|   mean| 493.51691715639276|
| stddev|   75.1074256896105|
|    min| 298.76200786180766|
|    max|  744.2218671047146|
+-------+-------------------+



In [62]:
lr = LinearRegression(labelCol = "Yearly Amount Spent")

In [65]:
lr_model = lr.fit(training_data)

In [66]:
test_results = lr_model.evaluate(test_data)

In [68]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|  10.43177320379084|
| -6.306033871914053|
| -1.025333118035519|
| 22.041163930737866|
| 3.8305745220067138|
|  3.465979490713721|
| -5.804258182896888|
| -9.376658953088452|
|-14.432705946059514|
| -7.347145082722477|
| -6.235192014660186|
|-3.6917656935481773|
|-17.745853476353943|
|-1.9199616606077825|
| -17.35352179768489|
| 11.579386412875067|
| -5.721445428876564|
| -17.93539983155756|
|-4.7668892299856225|
|  17.01839948744714|
+-------------------+
only showing top 20 rows



In [69]:
test_results.rootMeanSquaredError

9.575775055746668

In [70]:
test_results.r2

0.9836416478904471

In [71]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [72]:
unlabeled_data = test_data.select('features')

In [73]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.7377203726281...|
|[31.0613251567161...|
|[31.2606468698795...|
|[31.2834474760581...|
|[31.3584771924370...|
|[31.3662121671876...|
|[31.5257524169682...|
|[31.5261978982398...|
|[31.5741380228732...|
|[31.7207699002873...|
|[31.7242025238451...|
|[31.8124825597242...|
|[31.8164283341993...|
|[31.8627411090001...|
|[31.9048571310136...|
|[31.9262720263601...|
|[31.9453957483445...|
|[31.9563005605233...|
|[31.9673209478824...|
|[32.0180740106320...|
+--------------------+
only showing top 20 rows



In [74]:
prediction = lr_model.transform(unlabeled_data)

In [75]:
prediction.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.7377203726281...|451.34896899243904|
|[31.0613251567161...|493.86149192981566|
|[31.2606468698795...| 422.3519643749869|
|[31.2834474760581...| 569.7399254949296|
|[31.3584771924370...| 491.3453759274687|
|[31.3662121671876...| 427.1229030657712|
|[31.5257524169682...|449.76988499277877|
|[31.5261978982398...|418.47118514542626|
|[31.5741380228732...| 558.8419781066464|
|[31.7207699002873...| 546.1220785607454|
|[31.7242025238451...| 509.6230793026207|
|[31.8124825597242...| 396.5021106773454|
|[31.8164283341993...| 518.8683449800103|
|[31.8627411090001...| 558.2181028346545|
|[31.9048571310136...|491.30337922050103|
|[31.9262720263601...|380.62554703145133|
|[31.9453957483445...| 662.7413693665285|
|[31.9563005605233...| 565.0613315787564|
|[31.9673209478824...| 450.5167304696379|
|[32.0180740106320...|340.76471125786816|
+--------------------+------------